# Instructions

To run this report, you will first need to down load the following reports
* Aula report from snowflake -[latest SQL May23 Semester](https://livecoventryac.sharepoint.com/:w:/t/PlatformTeam/EdTeVzQu3BhHq8Wftzw-3ZkBSEIqRHrKJJbhgTmFBpjCOg?e=Tl14dx)
* FutureLearn learner report - Go to leaning manager and select Learner reports

Both files must be placed in the [PT Reporting/Jupyter Notebooks(Python) Reporting/SEC Report/Data](https://livecoventryac.sharepoint.com/:f:/t/PlatformTeam/EkJXF1VGYi9OnHshquJOYD0BPdj3TeujYX5RVTbXDH8PYA?e=xbRpHX) folder  

The import link for the Aula report can be found here, and must be updated to match your file - [Import Aula report](#aula-report)    

The import link for the FutureLearn learner report can be found here, and must be updated to match your file - [Import FL Report](#FL_report)  

The filter for weekly reporting can be amended here and must be changed weekly - [Filter last step visited date](#cut-off-date) 
The semester run start date can be amended here - [Filter run start date](#run-start-date) 

The filter for weekly reporting can be amended here and must be changed weekly - [Filter last step visited date](#cut-off-date) 


Once everything is set up and you are ready to run the report, save the notebook and the click on the 'kernel' tab on the menu bar and select 'Resart & run all'. The excel report will be availibale to download at the end of the notebook.

# Import libraries

In [ ]:
pip install xlsxwriter

In [ ]:
import pandas as pd
import numpy as np
import xlsxwriter
from datetime import datetime, date
from IPython.display import display, FileLink

# Import Aula report
<a id="aula-report"></a>

In [ ]:
# Amend file path below to location of aula report
aula_report = pd.read_csv('./Data/Aula report.csv', dtype={'STUDENT_ID':'str'})
aula_report.rename(columns={'STUDENT_ID':'learner_identifier'},inplace=True)
aula_report

# Import FL Learner report csv
<a id="FL_report"></a>

In [ ]:
# Amend file path below to location of FL learner report
df = pd.read_csv('./Data/FLLearner_Report_19.06.2023.csv', dtype={'run_code':'str', 'course_code':'str'})  # changes datatype on run code and course code to string
df

### Filter run start dates before this semester’s cut-off date
<a id="run-start-date"></a>

In [ ]:
# Change date each semester
df = df[(df['run_start_date'] > '2023-03-31')]

### Split date of last step visit to yyyy-mm-dd

In [ ]:
df = df.copy()
df.loc[:, 'date_of_last_step_visit'] = pd.to_datetime(df['date_of_last_step_visit'], errors='coerce').dt.date
df = df.reset_index(drop=True)
df


In [ ]:
# Tidy run code to remove course run number i.e 7050MHR-3_2324MAYAUG becomes 7050MHR_2324MAYAUG
df['run_code'] = df['run_code'].replace('-[^_]*', '', regex=True)
df

### Copy of enhanced learner report (elr) and add 'ProgramSID, Degree' columns

In [ ]:
elr = df
elr.insert(0,"ProgramSID", "")
elr.insert(1,"Degree", "")
elr

### Check N/As

In [ ]:
elr['run_code'].isnull().sum()


In [ ]:
# Strip leading and trailing whitespace from run code and check unique values
elr['run_code'] = elr['run_code'].str.strip()
elr['run_code'].unique()


### Drop N/As

In [ ]:
elr = elr.dropna(subset=['run_code'])
elr = elr.reset_index(drop=True)
elr


In [ ]:
elr['run_code'].isnull().sum()

In [ ]:
# Check unique values to see is 'nan' removed
elr['run_code'].unique()

### Lookup Degree code into elr

In [ ]:
elr_join_deg = pd.merge(elr,aula_report[['learner_identifier','DEGREE_COURSE']],
                        on='learner_identifier',
                        how='left').drop_duplicates()
elr_join_deg['Degree'] = elr_join_deg['DEGREE_COURSE']
elr_join_deg = elr_join_deg.reset_index(drop=True)
elr_join_deg

### Check N/As

In [ ]:
# Stip leading and trailng whitespace from degree column
elr_join_deg['Degree'] = elr_join_deg['Degree'].str.strip()
elr_join_deg['Degree'].unique()

In [ ]:
elr_join_deg['Degree'].isnull().sum()

### Filter out any staff emails

In [ ]:
pattern = '[a-zA-Z]{2}[0-9]{4}@coventry.ac.uk'
filtered_staff_email = elr_join_deg[elr_join_deg['learner_identifier'].str.contains(pattern)]

In [ ]:
filtered_staff_email

In [ ]:
elr_join_deg = elr_join_deg[~elr_join_deg['learner_identifier'].str.contains(pattern)].reset_index(drop=True)
elr_join_deg.shape


### Populate Program SID column

In [ ]:
elr_join_deg['ProgramSID'] = elr_join_deg['run_code'] + elr_join_deg['learner_identifier']
# Stip leading and trailng whitespace from programSID column
elr_join_deg['ProgamSID'] = elr_join_deg['ProgramSID'].str.strip()
elr_join_deg

In [ ]:
# Create final copy of FL learner report
FL_Learner_report = elr_join_deg.copy()

## Matching Program SIDs between Aula report and FL Report

In [ ]:
aula_report_2 = aula_report
aula_report_2['Match']=''
aula_report_2["ProgramSID"] = aula_report_2["EXTERNAL_ID"] + aula_report_2["learner_identifier"]
psid = aula_report_2.pop('ProgramSID')
  
# insert column using insert(position,column_name,
aula_report_2.insert(0, 'ProgramSID', psid)
aula_report_2['ProgramSID'] = aula_report_2['ProgramSID'].str.strip()
aula_report_2

### Lookup into Aula report match column (true/false)

In [ ]:
aula_report_2['Match'] = aula_report_2['ProgramSID'].isin(elr_join_deg['ProgramSID'])
aula_report_2

In [ ]:
false_count = aula_report_2['Match'].value_counts()[False]
false_count

# Creating the LTI not used sheet

In [ ]:
lti_not_used_draft = aula_report_2[aula_report_2['Match'] == False]
lti_not_used_draft

In [ ]:
# identify APHE and Apprentiship modules 
aphe_pattern = '[0-9]{4}CRB'
filtered_report_match = lti_not_used_draft[lti_not_used_draft['EXTERNAL_ID'].str.contains(aphe_pattern) | lti_not_used_draft['EXTERNAL_ID'].str.startswith(('A', 'a'))].reset_index(drop=True)
filtered_report_match

In [ ]:
# Remove APHE and Apprentiship modules 
lti_not_used_draft = lti_not_used_draft[~lti_not_used_draft['EXTERNAL_ID'].str.contains(aphe_pattern) | lti_not_used_draft['EXTERNAL_ID'].str.startswith(('A', 'a'))].reset_index(drop=True)
lti_not_used_draft

In [ ]:
lti_not_used_draft.shape

In [ ]:
lti_not_used_draft.rename(columns={'learner_identifier':'STUDENT_ID', 'EXTERNAL_ID':'FL Program Code'},inplace=True)
lti_not_used = lti_not_used_draft[['FL Program Code', 'STUDENT_ID','FIRST_NAME', 'LAST_NAME', 'EMAIL', 'DEGREE_COURSE']]
lti_not_used

# Creaing the not accessed between dates sheet

### Filter learner report by start date <= to today 

In [ ]:
elr_join_deg['run_start_date'] = pd.to_datetime(elr_join_deg['run_start_date'])
today = datetime.now().date()
not_accessed_draft = elr_join_deg[elr_join_deg['run_start_date'] <= pd.Timestamp(today)]
not_accessed_draft

In [ ]:
# identify APHE and Apprentiship modules 
filtered_not_accessed = not_accessed_draft[not_accessed_draft['run_code'].str.contains(aphe_pattern) | not_accessed_draft['run_code'].str.startswith(('A', 'a'))].reset_index(drop=True)
filtered_not_accessed

In [ ]:
# remove APHE and Apprentiship modules 
not_accessed = not_accessed_draft[~not_accessed_draft['run_code'].str.contains(aphe_pattern) | not_accessed_draft['run_code'].str.startswith(('A', 'a'))].reset_index(drop=True)
not_accessed

In [ ]:
not_accessed['learner_identifier'].nunique()

## Filter by date 
<a id="cut-off-date"></a>

In [ ]:
# Cut off date changed each week 
cut_off_date = pd.to_datetime('2023-06-13').date()

not_accessed = not_accessed[
    (not_accessed['date_of_last_step_visit'] < cut_off_date) |
    (pd.isnull(not_accessed['date_of_last_step_visit']))
]

not_accessed['date_of_last_step_visit'] = pd.to_datetime(not_accessed['date_of_last_step_visit'])

not_accessed['date_of_last_step_visit'].fillna(pd.to_datetime('1900-01-01'), inplace=True)
not_accessed

In [ ]:
req_columns = ['FIRST_NAME', 'LAST_NAME','EMAIL', 'learner_identifier']
not_accessed_merged = not_accessed.merge(aula_report_2[req_columns], on='learner_identifier', how='left')
not_accessed_merged.drop_duplicates()
not_accessed_merged

### Not accessed pivot table

In [ ]:
pivot_table = pd.pivot_table(not_accessed_merged, values='date_of_last_step_visit', index=['learner_identifier', 'FIRST_NAME', 'LAST_NAME', 'EMAIL'], aggfunc='max')
pivot_table['date_of_last_step_visit'] = pivot_table['date_of_last_step_visit'].replace(pd.to_datetime('1900-01-01'), '')
not_accessed_pivot_table = pivot_table.reset_index()  # Reset the index to make 'learner_identifier', 'FIRST_NAME', 'LAST_NAME', 'EMAIL' as separate columns
not_accessed_pivot_table = not_accessed_pivot_table[['learner_identifier', 'date_of_last_step_visit','FIRST_NAME', 'LAST_NAME', 'EMAIL']]  
not_accessed_pivot_table = not_accessed_pivot_table.sort_values(by='LAST_NAME')  # Sort the table by 'LAST_NAME' column
not_accessed_pivot_table = not_accessed_pivot_table.reset_index(drop=True) 
not_accessed_pivot_table



In [ ]:
learner_report = elr.copy()
learner_report['date_of_last_step_visit'] = pd.to_datetime(learner_report['date_of_last_step_visit'])
learner_report['date_of_last_step_visit'].fillna(pd.to_datetime('1900-01-01'), inplace=True)
learner_report

# Creating a list of student emails

In [ ]:
learner_report_emails = learner_report[learner_report['learner_identifier'].str.contains('@')].reset_index(drop=True)
learner_report_emails

## Pivot table of student emails with max 'date of last step visit'

In [ ]:
email_pivot_table = pd.pivot_table(learner_report_emails, values='date_of_last_step_visit', index=['learner_identifier'], aggfunc='max')
email_pivot_table['date_of_last_step_visit'] = email_pivot_table['date_of_last_step_visit'].replace(pd.to_datetime('1900-01-01'), '')
email_pivot_table = email_pivot_table.reset_index()  # Reset the index to make 'learner_identifier', 'FIRST_NAME', 'LAST_NAME', 'EMAIL' as separate columns

student_emails = email_pivot_table[['learner_identifier', 'date_of_last_step_visit']]  
student_emails = student_emails.sort_values(by='learner_identifier')  # Sort the table by 'LAST_NAME' column
student_emails = student_emails.reset_index(drop=True)
student_emails.rename(columns={'learner_identifier':'Students email'},inplace=True)
student_emails

# Creating a list of student IDs

In [ ]:
learner_report_sid = learner_report[learner_report['learner_identifier'].str.isdigit()].reset_index(drop=True)
learner_report_sid

## Pivot table of student IDs with max 'date of last step visit'


In [ ]:
sid_pivot_table = pd.pivot_table(learner_report_sid, values='date_of_last_step_visit', index=['learner_identifier'], aggfunc='max')
sid_pivot_table['date_of_last_step_visit'] = sid_pivot_table['date_of_last_step_visit'].replace(pd.to_datetime('1900-01-01'), '')
sid_pivot_table = sid_pivot_table.reset_index()  # Reset the index to make 'learner_identifier', 'FIRST_NAME', 'LAST_NAME', 'EMAIL' as separate columns

sid = sid_pivot_table[['learner_identifier', 'date_of_last_step_visit']]  
sid = sid.sort_values(by='learner_identifier')  # Sort the table by 'LAST_NAME' column
sid = sid.reset_index(drop=True)
sid.rename(columns={'learner_identifier':'SID'},inplace=True)
sid

### Output to Excel sheet

In [ ]:
# Convert date columns to datetime format
FL_Learner_report['date_of_last_step_visit'] = pd.to_datetime(FL_Learner_report['date_of_last_step_visit'])
not_accessed_pivot_table['date_of_last_step_visit'] = pd.to_datetime(not_accessed_pivot_table['date_of_last_step_visit'])
sid['date_of_last_step_visit'] = pd.to_datetime(sid['date_of_last_step_visit'])
student_emails['date_of_last_step_visit'] = pd.to_datetime(student_emails['date_of_last_step_visit'])


# Convert date columns to date type and extract only the date portion
FL_Learner_report['date_of_last_step_visit'] = FL_Learner_report['date_of_last_step_visit'].dt.date
not_accessed_pivot_table['date_of_last_step_visit'] = not_accessed_pivot_table['date_of_last_step_visit'].dt.date
sid['date_of_last_step_visit'] = sid['date_of_last_step_visit'].dt.date
student_emails['date_of_last_step_visit'] = student_emails['date_of_last_step_visit'].dt.date

In [ ]:
file_path = 'SEC_Report.xlsx'

# Save the DataFrames to the Excel workbook
with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    FL_Learner_report.to_excel(writer, sheet_name='FL Learner Report', index=False)
    lti_not_used.to_excel(writer, sheet_name='LTI link not used', index=False)
    not_accessed_pivot_table.to_excel(writer, sheet_name='FL Not Accessed', index=False)
    sid.to_excel(writer, sheet_name='Students IDs', index=False)
    student_emails.to_excel(writer, sheet_name='Student emails', index=False)

# Display the file path as a link
display(FileLink(file_path))